In [1]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

import gspread 
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('alberta-covid-19-dashboard-b0928e3c242b.json', scope) 
gc = gspread.authorize(credentials)

spreadsheet_key = '19fGEpHtSUHF2WR5IVaVdm0UD2ftVicneMTiXofXt24E'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheets()[0]
table = worksheet.get_all_values()

from scipy.optimize import minimize, curve_fit

%matplotlib inline

In [2]:
def easy_subplots():
    return make_subplots(specs=[[{"secondary_y":True}]])

def rollingAverage(_list, _delta=3):
    _newList = []
    for i in range(_delta,len(_list)-_delta):
        _newList.append(np.array(_list[i-_delta:i+_delta+1]).mean())
    return np.array(_newList)

In [3]:
df = pd.DataFrame(data=table,columns=['Date','Number of Cases'])#pd.read_csv('COVID-19.csv',header=None,names=['Date','Number of Cases'])
df['Date'] = df['Date'].apply(lambda d:pd.to_datetime(d))
df['Number of Cases'] = df['Number of Cases'].astype(int)

scrapedData = pd.DataFrame(data=book.worksheets()[1].get_all_values())
newDate = pd.to_datetime(scrapedData[0].iloc[2])
nCases = int(scrapedData[2].iloc[5])
if (newDate>df['Date'].iloc[-1]):
    df = df.append(pd.DataFrame({'Date':[newDate],'Number of Cases':[nCases]})).reset_index(drop=True)
    worksheet.update_cell(len(df),1,newDate.date().strftime('%Y-%m-%d'))
    worksheet.update_cell(len(df),2,nCases)
    #row = [newDate.date().strftime('%Y-%m-%d'),nCases]
    #worksheet.insert_row(row,len(table)+1)
    
testData = pd.DataFrame(data=book.worksheets()[2].get_all_values(),columns=['Date','Number of Tests'])
testData['Date'] = testData['Date'].apply(lambda d:pd.to_datetime(d))
testData['Number of Tests'] = testData['Number of Tests'].astype(int)
testData['Percent Increase'] = 0.0
newDateTests = pd.to_datetime(scrapedData[0].iloc[3])
nTests = int(scrapedData[6].iloc[4])
if (newDateTests>testData['Date'].iloc[-1]):
    testData = testData.append(pd.DataFrame({'Date':[newDateTests], 'Number of Tests':[nTests]})).reset_index(drop=True)
    book.worksheets()[2].update_cell(len(testData),1,newDateTests.date().strftime('%Y-%m-%d'))
    book.worksheets()[2].update_cell(len(testData),2,nTests)
for i in range(1,len(testData)):
    delta = testData['Number of Tests'][i] - testData['Number of Tests'][i-1]
    testData['Percent Increase'][i] = 100. * float(delta / testData['Number of Tests'][i-1])

df['Uncertainty'] = np.sqrt(df['Number of Cases'])
df['Delta'] = 0
df['Percent Increase'] = 0.
df['Increase Uncertainty'] = 0.
for i in range(1,len(df)):
    df['Delta'][i] = df['Number of Cases'][i] - df['Number of Cases'][i-1]
    if df['Date'][i] > pd.to_datetime('2020-03-08'):
        df['Percent Increase'][i] = 100. * float(df['Delta'][i]) / df['Number of Cases'][i-1]
        df['Increase Uncertainty'][i] = df['Percent Increase'][i] * np.sqrt( (np.square(df['Uncertainty'][i])+np.square(df['Uncertainty'][i-1]))/np.square(df['Delta'][i]) + 1/df['Number of Cases'][i-1] )
df = df.set_index('Date')

C:\Users\joejw\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\joejw\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\joejw\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\joejw\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A 

In [4]:
gridlines = 5

def calculateRange(y1min, y1max, y2min, y2max):
    y1_max = y1max
    y1_min = y1min

    y1_range = y1_max - y1_min
    y1_len = len(str(y1_range))
    y1_pow10_divisor = np.power(10, y1_len-1)
    y1_firstdigit = np.floor(y1_range / y1_pow10_divisor)
    y1_max_base = y1_pow10_divisor * y1_firstdigit
    y1_dtick = y1_max_base / gridlines

    y2_max = y2max#760
    y2_min = y2min#0

    y2_range = y2_max - y2_min
    y2_len = len(str(y2_range))
    y2_pow10_divisor = np.power(10, y2_len-1)
    y2_firstdigit = np.floor(y2_range / y2_pow10_divisor)
    y2_max_base = y2_pow10_divisor * y2_firstdigit
    y2_dtick = y2_max_base / gridlines

    y1_dtick_ratio = y1_range / y1_dtick
    y2_dtick_ratio = y2_range / y2_dtick

    global_dtick_ratio = np.max([y1_dtick_ratio, y2_dtick_ratio])

    y1_negative_ratio = np.abs(y1_min/y1_range) * global_dtick_ratio
    y2_negative_ratio = np.abs(y2_min/y2_range) * global_dtick_ratio
    
    if y1_min<0 or y2_min<0:
        global_negative_ratio = np.max([y1_negative_ratio,y2_negative_ratio]) + 0.1
    else:
        global_negative_ratio = 0

    y1_range_min = global_negative_ratio * y1_dtick * -1
    y2_range_min = global_negative_ratio * y2_dtick * -1

    y1_positive_ratio = np.abs(y1_max / y1_range) * global_dtick_ratio
    y2_positive_ratio = np.abs(y2_max / y2_range) * global_dtick_ratio

    global_positive_ratio = np.max([y1_positive_ratio,y2_positive_ratio]) + 0.1

    y1_range_max = global_positive_ratio * y1_dtick
    y2_range_max = global_positive_ratio * y2_dtick
    
    return y1_range_min, y1_range_max, y1_dtick, y2_range_min, y2_range_max, y2_dtick

y1_range_min, y1_range_max, y1_dtick, y2_range_min, y2_range_max, y2_dtick = calculateRange(0,105,0,760)

In [5]:
startDate = '2020-03-09'
startDate2 = '2020-03-08'

In [6]:
df['Days Since March 8th'] = df.reset_index()['Date'].apply(lambda d:(d-pd.to_datetime('2020-03-08')).days).values

In [7]:
def fitExp(x, a, b):
    return a*np.exp(b*x)

popt, pcov = curve_fit(fitExp,df.loc['2020-03-11':'2020-03-20']['Days Since March 8th'],df.loc['2020-03-11':'2020-03-20']['Number of Cases'])
popt2, pcov2 = curve_fit(fitExp,df.loc['2020-03-20':'2020-03-24']['Days Since March 8th'],df.loc['2020-03-20':'2020-03-24']['Number of Cases'])

#fit = np.polyfit(df.loc['2020-03-10':'2020-03-20']['Days Since March 8th'].values,df.loc['2020-03-10':'2020-03-20']['Number of Cases'].values,2)
#fit

In [20]:
logScale = widgets.Checkbox(
    description='Log Scale',
    value=False
)

enlarge = widgets.Checkbox(
    description='Enlarge Plot',
    value=False
)

predictOutTo = widgets.IntSlider(value=3,
                                 min=0,
                                 max=10,
                                 step=1,
                                 description='Days to extend the prediction out from today by: ',
                                 style={'description_width':'initial'},
                                 layout=Layout(width='50%'),
                                 continuous_update=True)
fig = go.FigureWidget(easy_subplots())

maxDays = int(np.floor((pd.to_datetime('today')-pd.to_datetime('2020-03-08')).days))+predictOutTo.value
x = np.linspace(0,maxDays,10*maxDays+1)
y = fitExp(x,*popt)
startIndex = int(np.floor(12/(x[1]-x[0])))
y2 = fitExp(x[startIndex:],*popt2)

fig.add_trace(go.Bar(x=df.loc[startDate:].index,
                         y=df.loc[startDate:]['Percent Increase'],
                         #mode='markers',
                         marker=dict(color='crimson'),
                         name='Case % Increase',
#                          error_y=dict(type='data',
#                                       array=df.loc[startDate:]['Increase Uncertainty'],
#                                       visible=True,
#                                       color='maroon')
                        ),
              secondary_y = False)

fig.add_trace(go.Bar(x=testData['Date'].iloc[1:],
                         y=testData['Percent Increase'].iloc[1:],
                         #mode='markers',
                         marker=dict(color='olive'),
                         name='Test % Increase',
#                          error_y=dict(type='data',
#                                       array=df.loc[startDate:]['Increase Uncertainty'],
#                                       visible=True,
#                                       color='maroon')
                        ),
              secondary_y = False)

fig.add_trace(go.Scatter(x=df.loc[startDate2:].index,
                         y=df.loc[startDate2:]['Number of Cases'],
                         mode='markers',
                         marker=dict(color='navy',symbol='circle',line_color='white'),
                         name='# of Cases',
#                          error_y=dict(type='data',
#                                       array=df.loc[startDate2:]['Uncertainty'],
#                                       visible=True)
                        ),
              secondary_y = True)

fig.add_trace(go.Scatter(x=np.arange(pd.to_datetime(startDate2),pd.to_datetime(startDate2)+pd.Timedelta(np.floor((pd.to_datetime('today')-pd.to_datetime('2020-03-08')).days)+predictOutTo.value,'D'),pd.Timedelta(x[1]-x[0],'D')),
                         y=y,
                         name='Exp Fit',
                         line=dict(color='navy',
                                   dash='dot')
                        ),
              secondary_y=True
             )
fig.add_trace(go.Scatter(x=np.arange(pd.to_datetime('2020-03-20'),pd.to_datetime('2020-03-08')+pd.Timedelta(x[-1],'D'),pd.Timedelta(x[1]-x[0],'D')),
                         y=y2,
                         name='New Exp Fit',
                         line=dict(color='navy',
                                   dash='dash')
                        ),
              secondary_y=True
             )

fig.update_xaxes(showgrid=False)
fig.update_layout(title='Alberta COVID-19 Cases',plot_bgcolor='lightgrey')
y1_range_min, y1_range_max, y1_dtick, y2_range_min, y2_range_max, y2_dtick = calculateRange(0,105,0,int(y2[-1]*1.1))
fig.update_yaxes(title_text="% Increase", color='crimson',range=[y1_range_min,y1_range_max],dtick=y1_dtick,secondary_y = False)
fig.update_yaxes(title_text="Number of Cases", color='navy',range=[y2_range_min,y2_range_max],type='linear',tickformat='f',dtick=y2_dtick,secondary_y = True,gridcolor='white')
def response(change):
    global fig
    
    if enlarge.value:
        fig.update_layout(height=fig.layout.width/2.1)
    else:
        fig.update_layout(height=fig.layout.width/4)
    
    maxDays = int(np.floor((pd.to_datetime('today')-pd.to_datetime('2020-03-08')).days))+predictOutTo.value
    x = np.linspace(0,maxDays,10*maxDays+1)
    y = fitExp(x,*popt)
    startIndex = int(np.floor(12/(x[1]-x[0])))
    y2 = fitExp(x[startIndex:],*popt2)
    y1_range_min, y1_range_max, y1_dtick, y2_range_min, y2_range_max, y2_dtick = calculateRange(0,105,0,int(y2[-1]*1.1))
    
    scatter = fig.data[3]
    scatter.x=np.arange(pd.to_datetime(startDate2),pd.to_datetime(startDate2)+pd.Timedelta(np.floor((pd.to_datetime('today')-pd.to_datetime('2020-03-08')).days)+predictOutTo.value,'D'),pd.Timedelta(x[1]-x[0],'D'))
    scatter.y=y
    
    scatter2 = fig.data[4]
    scatter2.x=np.arange(pd.to_datetime('2020-03-20'),pd.to_datetime('2020-03-08')+pd.Timedelta(x[-1],'D'),pd.Timedelta(x[1]-x[0],'D'))
    scatter2.y=y2
    
    fig.update_yaxes(title_text="% Increase", color='crimson',range=[y1_range_min,y1_range_max],dtick=y1_dtick,secondary_y = False)
    
    if logScale.value:
        fig.update_yaxes(title_text="Number of Cases", color='navy',range=None,tick0=np.log(0.1),dtick=None,type='log',tickformat='f',secondary_y = True,gridcolor='blue')
    else:
        fig.update_yaxes(title_text="Number of Cases", color='navy',range=[y2_range_min,y2_range_max],tick0=y2_range_min,type='linear',tickformat='f',dtick=y2_dtick,secondary_y = True,gridcolor='white')


In [21]:
logScale.observe(response, names='value')
predictOutTo.observe(response, names='value')
enlarge.observe(response, names='value')

container = widgets.HBox([enlarge, logScale, predictOutTo])
widgets.VBox([container,fig])

In [70]:
logScale2 = widgets.Checkbox(
    description='Log Scale',
    value=False
)

enlarge2 = widgets.Checkbox(
    description='Enlarge Plot',
    value=False
)

rollingAvgDelta = widgets.IntSlider(value=3,
                                 min=1,
                                 max=11,
                                 step=2,
                                 description='Moving Average Length of Days: ',
                                 style={'description_width':'initial'},
                                 layout=Layout(width='50%'),
                                 continuous_update=True)

fig2 = go.FigureWidget()
delta = 1
fig2.add_trace(go.Scatter(x=df[startDate2:].iloc[delta:-delta]['Number of Cases'],
                          y=rollingAverage(df[startDate2:]['Delta'],_delta=delta),
                          mode='lines+markers'
                        )
             )
endDate = df[startDate2:].iloc[delta:-delta].index[-1]
fig2.update_layout(plot_bgcolor='lightgrey',
                   title='Trajectory of COVID-19 Confirmed Cases in Alberta (%d/%d/%d)'%(endDate.month,
                                                                                         endDate.day,
                                                                                         endDate.year)
                  )
fig2.update_yaxes(tickformat='f',title='Number of New Cases')
fig2.update_xaxes(tickformat='f',title='Total Cases')

def updateSecondPlot(change):
    if enlarge2.value:
        fig2.update_layout(height=fig2.layout.width/2.1)
    else:
        fig2.update_layout(height=fig2.layout.width/4)
    
    delta = int((rollingAvgDelta.value-1)/2)
    scatter = fig2.data[0]
    if delta>=1:
        scatter.x = df[startDate2:].iloc[delta:-delta]['Number of Cases']
        scatter.y = rollingAverage(df[startDate2:]['Delta'],_delta=delta)
        endDate = df[startDate2:].iloc[delta:-delta].index[-1]
    else:
        scatter.x = df[startDate2:]['Number of Cases']
        scatter.y = df[startDate2:]['Delta']
        endDate = df[startDate2:].index[-1]
        
    fig2.update_layout(title='Trajectory of COVID-19 Confirmed Cases in Alberta (%d/%d/%d)'%(endDate.month,
                                                                                             endDate.day,
                                                                                             endDate.year)
                  )
    
    if logScale2.value:
        fig2.update_xaxes(type='log',tickformat='f')
        fig2.update_yaxes(type='log',tickformat='f')
    else:
        fig2.update_xaxes(type='linear',tickformat='f')
        fig2.update_yaxes(type='linear',tickformat='f')


In [71]:
rollingAvgDelta.observe(updateSecondPlot, names='value')
logScale2.observe(updateSecondPlot, names='value')
enlarge2.observe(updateSecondPlot, names='value')

container2 = widgets.HBox([enlarge2,logScale2,rollingAvgDelta])
widgets.VBox([container2,fig2])

In [77]:
enlarge3 = widgets.Checkbox(
    description='Enlarge Plot',
    value=False
)

fig3 = go.FigureWidget(easy_subplots())

y1min, y1max, y1dtick, y2min, y2max, y2dtick = calculateRange(0,df['Number of Cases'].max(),0,testData['Number of Tests'].max())

fig3.update_yaxes(title='Number of Confirmed Cases',range=[y1min,y1max],dtick=y1dtick,secondary_y=False,color='navy')
fig3.update_yaxes(title='Number of Tests Performed',range=[y2min,y2max],dtick=y2dtick,secondary_y=True,color='crimson')

fig3.update_layout(plot_bgcolor='lightgrey',showlegend=False)

fig3.add_trace(go.Scatter(x=df.index,
                          y=df['Number of Cases'],
                          name='Cases',
                          line=dict(color='navy')
                         ),
               secondary_y=False)

fig3.add_trace(go.Scatter(x=testData['Date'],
                          y=testData['Number of Tests'],
                          name='Tests',
                          line=dict(color='crimson'),
                          marker=dict(color='crimson')
                         ),
               secondary_y=True)

def updateThirdPlot(change):
    if enlarge3.value:
        fig3.update_layout(height=fig3.layout.width/2.1)
    else:
        fig3.update_layout(height=fig3.layout.width/4)

In [78]:
enlarge3.observe(updateThirdPlot, names='value')

container3 = widgets.HBox([enlarge3])
widgets.VBox([container3,fig3])

    'data':…